In [1]:
# This runs a bit faster, but not a lot
# it may just be best to load the whole model onto the gpu 
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import os
import math
import time

In [2]:
print(torch.cuda.is_available())

True


In [3]:
# dataset that directly loads the file into memory and then retrieves data as needed
# this helps deal with the file read bottlenecks, but the data has to be transformed and then loaded to the gpu
class BowlingDataset(Dataset):
    def __init__(self, fileName):
        self.f = open(fileName, "rb")
        self.length = int(os.stat(fileName).st_size/27)
        #if self.length > 1500000:
        #    self.length = 1500000
        self.f.seek(0)
        self.fileData = self.f.read()
        self.f.close()
    def __len__(self):
        return self.length
    
    def __getitem__(self, idx):
        gameData = self.fileData[idx * 27: idx * 27 + 27]
        tempArray = []
        for x in range(25):
            tempArray += [v for v in format(gameData[x], "08b")]
        finalScore = gameData[-1] * 256 + gameData[-2]
        inputArray = [float(v) for v in tempArray][:120]
        output = finalScore
        return torch.tensor(inputArray[:]), torch.tensor(float(output))

In [4]:
class TestModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.relu_stack = torch.nn.Sequential(
            torch.nn.Linear(120, 1024),
            torch.nn.ReLU(),
            torch.nn.Linear(1024, 512),
            torch.nn.ReLU(),
            torch.nn.Linear(512, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 1)
        )
        
    def forward(self, x):
        self.logits = self.relu_stack(x)
        self.output = torch.nn.Sigmoid()(self.logits)
        return self.output * 300

In [11]:
def train_loop(dataloader, modelList, lossFnList, optimizerList):
    size = len(dataloader.dataset) # get number of samples
    numModels = len(modelList)
    totalBatches = len(dataloader)
    predictionList = [0] * len(modelList)
    lossList = [0] * len(modelList)
    for model in modelList:
        model.train() # need to look into what this exactly does
    startTime = time.time()
    for batchNum, (x, y) in enumerate(dataloader):
        #grab model input and label as tensors on the gpu
        xTensor = x.cuda()
        yTensor = y.cuda()
        
        # the point of all these consecutive for loops is that the cuda operations should be async
        # and then automatically synced up when the result tensor is needed
        
        # zero out gradients of each optimizer
        for x in range(numModels):
            optimizerList[x].zero_grad()
            
        # get predictions
        for x in range(numModels):
            predictionList[x] = modelList[x](xTensor)
        
        #compute losses
        for x in range(numModels):
            lossList[x] = lossFnList[x](predictionList[x].squeeze(1), yTensor)
        
        #run backprogagation
        for x in range(numModels):
            lossList[x].backward()
        
        #update weights
        for x in range(numModels):
            optimizerList[x].step()
        
        if not batchNum % 1000:
            for x in range(numModels):
                print(f"Model{x} -- loss: {lossList[x].item():.2f}\tbatch num: {batchNum}/{totalBatches}")
                print(f"took {time.time() - startTime} seconds")
            startTime = time.time()

In [6]:
def test_loop(dataloader, modelList, loss_fn):
    for model in modelList:
        model.eval() # need to look into what this exactly does
    size = len(dataloader.dataset)
    numBatches = len(dataloader)
    test_loss = 0
    total_losses = [0] * len(modelList)
    
    with torch.no_grad():
        for x, y in dataloader:
            xTensor = x.cuda()
            yTensor = y.cuda()
            for x in range(len(modelList)):
                pred = model(xTensor)
                total_losses[x] += loss_fn(pred, yTensor).item()
    for x in range(len(modelList)):
        print(f"model{x} Test Set -- Average Loss: {total_losses[x]/numBatches}")

In [7]:
# HYPERPARAMS
batch_size = 64
learning_rate = 0.0005
epochs = 8

In [8]:
trainData = BowlingDataset("ScoreDetailDataset.txt")
trainDataLoader = DataLoader(trainData, batch_size=batch_size, shuffle=True, pin_memory=True) #pin memory doesnt do shit bc the memory has to be grabbed from a physical file
testData = BowlingDataset("ScoreDetailDatasetVSplit.txt")
testDataLoader = DataLoader(testData, batch_size=batch_size, shuffle=True)

In [12]:
model0 = TestModel().cuda()
optim0 = torch.optim.Adam(model0.parameters(), lr=0.001)
lossfn0 = torch.nn.MSELoss().cuda()

model1 = TestModel().cuda()
optim1 = torch.optim.Adam(model1.parameters(), lr=0.0005)
lossfn1 = torch.nn.MSELoss().cuda()

model2 = TestModel().cuda()
optim2 = torch.optim.Adam(model2.parameters(), lr=0.0001)
lossfn2 = torch.nn.MSELoss().cuda()

model3 = TestModel().cuda()
optim3 = torch.optim.SGD(model3.parameters(), lr=0.001)
lossfn3 = torch.nn.MSELoss().cuda()

modelList = [model0, model1, model2, model3]
optimList = [optim0, optim1, optim2, optim3]
lossfnList = [lossfn0, lossfn1, lossfn2, lossfn3]
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for t in range(epochs):
    startTime = time.time()
    print(f"Starting epoch {t}")
    train_loop(trainDataLoader, modelList, lossfnList, optimList)
    print(f"Epoch {t} took {time.time() - startTime} seconds")
    test_loop(testDataLoader, modelList, lossfnList[0])
print("Finished")

Starting epoch 0
Model0 -- loss: 3355.13	batch num: 0/62321
took 0.3431823253631592 seconds
Model1 -- loss: 3826.80	batch num: 0/62321
took 0.3431823253631592 seconds
Model2 -- loss: 4478.77	batch num: 0/62321
took 0.3431823253631592 seconds
Model3 -- loss: 4639.71	batch num: 0/62321
took 0.3431823253631592 seconds
Model0 -- loss: 227.69	batch num: 1000/62321
took 38.52545619010925 seconds
Model1 -- loss: 230.31	batch num: 1000/62321
took 38.526453495025635 seconds
Model2 -- loss: 258.45	batch num: 1000/62321
took 38.526453495025635 seconds
Model3 -- loss: 34462.34	batch num: 1000/62321
took 38.526453495025635 seconds
Model0 -- loss: 180.89	batch num: 2000/62321
took 39.35813307762146 seconds
Model1 -- loss: 200.59	batch num: 2000/62321
took 39.35908794403076 seconds
Model2 -- loss: 227.10	batch num: 2000/62321
took 39.35908794403076 seconds
Model3 -- loss: 36746.45	batch num: 2000/62321
took 39.35908794403076 seconds
Model0 -- loss: 247.10	batch num: 3000/62321
took 27.760605335235596

KeyboardInterrupt: 